In [ ]:
import torch
import einops
from jaxtyping import Float

from transformer_lens import HookedTransformer

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
batch_size = 10
seq_len = 50
random_tokens = torch.randint(1000, 10000, (batch_size, seq_len)).to(model.cfg.device)
repeated_tokens = einops.repeat(random_tokens, "batch seq_len -> batch (2 seq_len)")

In [ ]:
induction_score_store = torch.zeros((model.cfg.n_layers, model.cfg.n_heads), device=model.cfg.device)

In [ ]:
induction_score_store.shape

torch.Size([12, 12])

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
def induction_score_hook(act, hook):
    set_trace()
    induction_stripe = act.diagonal(dim1=-2, dim2=-1, offset=1-seq_len)
    set_trace()

In [ ]:
pattern_hook_names_filter = lambda name: name.endswith("pattern")

In [ ]:
repeated_tokens.shape

torch.Size([10, 100])

In [ ]:
model.run_with_hooks(
    repeated_tokens,
    return_type=None,
    fwd_hooks=[(
        pattern_hook_names_filter,
        induction_score_hook
    )]
)

> /var/folders/pf/g3nr86yn4j71vzzmv6knwdhr0000gp/T/ipykernel_1780/953945506.py(3)induction_score_hook()
      1 def induction_score_hook(act, hook):
      2     set_trace()
----> 3     induction_stripe = act.diagonal(dim1=-2, dim2=-1, offset=1-seq_len)
      4     set_trace()



ipdb>  seq_len


50


ipdb>  act.shape


torch.Size([10, 12, 100, 100])


ipdb>  q


In [ ]:
9 3 8 2 7 9 3 8 2 7

In [ ]:
12341234

In [ ]:
prompt = "123456789123"

In [ ]:
tokens = model.to_tokens(prompt)

In [ ]:
len(tokens[0])

6

In [ ]:
logits = model(tokens)

In [ ]:
def compute_induction_score(
    act,
    hook
):
    print(act.shape)

In [ ]:
pattern_hook_names_filter = lambda name: name.endswith("pattern")

In [ ]:
model.run_with_hooks(
    tokens,
    return_type=None,
    fwd_hooks=[(pattern_hook_names_filter, compute_induction_score)]
)

torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])
torch.Size([1, 12, 14, 14])


In [ ]:
logits.shape

torch.Size([1, 14, 50257])